In [1]:
import cv2
import json
import requests
import http.client
import urllib.request
import urllib.parse
import urllib.error
import base64

In [2]:
def analyze_frame(image_path, threshold):
    headers = {
        # Request headers
        'Content-Type': 'application/octet-stream',
        'Prediction-key': 'fe3f42c3c33449e8bba854e961cd72e0',
    }

    params = urllib.parse.urlencode({
        # Request parameters
        'iterationId': '',
        'application': '',
    })
    
    image = open(image_path, "rb").read()

    try:
        conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
        conn.request("POST", "/customvision/v1.1/Prediction/f9c2c85a-fdcb-4e96-a5a5-e085e58187eb/image?%s" % params, image, headers)
        response = conn.getresponse()
        analysis = json.loads(response.read())
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    dict_list = analysis['Predictions']
    for dictionary in dict_list:
        if dictionary['Probability'] >= threshold:
            return True
    return False

In [ ]:
camera = cv2.VideoCapture(0)
camera.set(cv2.CAP_PROP_FPS, 0.2)

In [4]:
#analysis every frame
while(True):
    ret, frame = camera.read()
    cv2.imshow('frame', frame)
    
    cv2.imwrite('capture.jpg', frame)
    alarm = analyze_frame('capture.jpg', 0.9)
    if alarm:
        print("ALARM")
    else:
        print("NO GUN DETECTED")
    
    key = cv2.waitKey(3)
    #ESC key press
    if key%256 == 27:
        break
        
camera.release()
cv2.destroyAllWindows()

NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED


KeyError: 'Predictions'

In [4]:
#analysis on key press
while(True):
    ret, frame = camera.read()
    cv2.imshow('frame', frame)
    
    key = cv2.waitKey(3)
    #ESC key press
    if key%256 == 27:
        break
    #SPACE key press
    elif key%256 == 32:
        cv2.imwrite('capture.jpg', frame)
        alarm = analyze_frame('capture.jpg', 0.9)
        if alarm:
            print("ALARM")
        else:
            print("NO GUN DETECTED")
        
camera.release()
cv2.destroyAllWindows()

NO GUN DETECTED
ALARM
ALARM
ALARM
ALARM
ALARM
ALARM
ALARM
ALARM
ALARM
ALARM
ALARM
ALARM
ALARM
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
ALARM
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
ALARM
ALARM
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
ALARM
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
ALARM
ALARM
NO GUN DETECTED
ALARM
ALARM
